## Experiment-04
### Amirreza Fosoul and Bithiah Yuan

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
import pyspark.sql.functions as f
import csv
spark = SparkSession.builder.appName('ex4').getOrCreate()

In [2]:
import string
import re
import time
from pyspark.sql.functions import *
from pyspark.sql.functions import split, udf, desc, concat, col, lit
import pyspark.sql.functions as f
from pyspark.sql.types import ArrayType, FloatType, StringType, IntegerType, DoubleType, StructType, StructField
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.window import Window
from pyspark.ml.linalg import SparseVector, VectorUDT, DenseVector
import scipy.sparse
from pyspark.ml.linalg import Vectors, _convert_to_vector, VectorUDT
import numpy as np
from pyspark.sql import SQLContext
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import math
import re

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [3]:
# Read user ratings into Dataframe
#user_df = spark.read.option("delimiter", ";").csv('./users_libraries.txt')
user_df = spark.read.option("delimiter", ";").csv('./example2.txt')
user_df = user_df.select(col("_c0").alias("userID"), col("_c1").alias("paperID"))
user_df_pre = user_df

user_df_pre = user_df_pre.withColumn("paperID", split(col("paperID"), ",").cast(ArrayType(IntegerType())).alias("paperID"))
user_df_pre.show()

user_df = user_df.select("userID", f.split("paperID", ",").alias("papers"), f.explode(f.split("paperID", ",")).alias("paperID"))
user_df = user_df.drop("papers")

# Get a dataframe of the distinct papers
d_paper = user_df.select("paperID").distinct()

# # Get the number of distinct papers
# num_papers = user_df.select("paperID").distinct().count()

# # Get the number of distinct users
# num_users = user_df.select("userID").distinct().count()


+--------------------+--------------------+
|              userID|             paperID|
+--------------------+--------------------+
|28d3f81251d94b097...|[3929762, 503574,...|
|d0c9aaa788153daea...|[2080631, 6343346...|
|f05bcffe7951de9e5...|[1158654, 478707,...|
|ca4f1ba4094011d9a...|            [278019]|
|d1d41a15201915503...|[6610569, 6493797...|
|f2f77383828ea6d39...|[943458, 238121, ...|
|9c883d02115400f7b...|[3509971, 3509965...|
|b656009a6efdc8b1a...|[771870, 181369, ...|
|cf9c7f356092c34be...|             [90558]|
|0f5cbb39410a9278f...|           [9344598]|
|d85f7d83f27b3f533...|[7610843, 3633347...|
|586c867a0688250ac...|[464760, 466011, ...|
|10fdfaf945d5c27ad...|           [2010550]|
|589b870a611c25fa9...|[1283233, 1305474...|
|90f1a3e6fcdbf9bc5...|[115945, 11733005...|
|7e070a9da96672e05...|           [1071959]|
|3b715ebaf1f8f81a1...|[4119394, 3378798...|
|488fb15e8c77f8054...|[1523301, 5281566...|
|3fdf355e59949c79d...|[7077220, 1289842...|
|c6b59086a0bbac141...|[2230995, 

In [5]:
# Read in the stopwords as a list
with open('./stopwords_en.txt') as file:
    stopwordList = file.read().splitlines()

# Read in records of paper information
#w_df = spark.read.csv('./papers.csv')
w_df = spark.read.csv('./example_paper.csv')
w_df = w_df.select("_c0", "_c13", "_c14")
w_df = w_df.select(col("_c0").alias("paperID"), col("_c13").alias("title"), col("_c14").alias("abstract"))
w_df = w_df.na.fill({'title': '', 'abstract': ''}) # to replace null values with empty string
# Get text from title and abstract
w_df = w_df.select(col("paperID"), concat(col("title"), lit(" "), col("abstract")).alias("words"))

# Transform the distinct paperIDs dataframe to a list
paper_list = list(d_paper.select('paperID').toPandas()['paperID'])
# Map each distinct paper into int
paper_list = list(map(int, paper_list))



In [6]:
# Function to call in udf
def unrated(papers):
    # Transform the list of distinct papers and the list of rated papers of each user to a set
    # Substract the two sets to get the list of unrated papers for each user
    # Transform back to list
    unrated = list(set(paper_list) - set(papers))
    
    return unrated


# udf to get a list of unrated papers with the length of rated papers for each user
get_unrated = udf(lambda x: unrated(x), ArrayType(IntegerType()))

# Add a new column of unrated papers for each user
unrated_df = user_df_pre.withColumn("unrated", get_unrated(user_df_pre.paperID))

unrated_df = unrated_df.drop("paperID")

unrated_df = unrated_df.withColumn("paperID", explode(unrated_df.unrated))

unrated_df = unrated_df.drop("unrated")

unrated_df.show()

+--------------------+-------+
|              userID|paperID|
+--------------------+-------+
|28d3f81251d94b097...|1277953|
|28d3f81251d94b097...|3153930|
|28d3f81251d94b097...|3153931|
|28d3f81251d94b097...|1388555|
|28d3f81251d94b097...|4186128|
|28d3f81251d94b097...|1814546|
|28d3f81251d94b097...|7499794|
|28d3f81251d94b097...| 921623|
|28d3f81251d94b097...| 970776|
|28d3f81251d94b097...| 311321|
|28d3f81251d94b097...|7499801|
|28d3f81251d94b097...| 118812|
|28d3f81251d94b097...| 290846|
|28d3f81251d94b097...|4067359|
|28d3f81251d94b097...| 688160|
|28d3f81251d94b097...|1118240|
|28d3f81251d94b097...|  81954|
|28d3f81251d94b097...|4460578|
|28d3f81251d94b097...| 340004|
|28d3f81251d94b097...|2363430|
+--------------------+-------+
only showing top 20 rows



## Exercise 3. 1 Vector representation for the papers

In [7]:
# Extracting words from the papers and keeping "-" and "_"
tokenizer = RegexTokenizer(inputCol="words", outputCol="tokens", pattern="[a-zA-Z-_]+", gaps=False) 
# Built-in tokenizer
tokenized = tokenizer.transform(w_df)
tokenized = tokenized.select("paperID", "tokens")

# udf to remove "-" and "_" from the tokens
remove_hyphen_udf = udf(lambda x: [re.sub('[-|_]', '', word) for word in x], ArrayType(StringType()))
# Apply udf to the tokens
df = tokenized.withColumn('tokens', remove_hyphen_udf(col('tokens')))

# udf to remove words less than 3 letters
remove_short_words = udf(lambda x: [item for item in x if len(item) >= 3], ArrayType(StringType()))
# Apply udf to the tokens
df = df.withColumn('tokens', remove_short_words(col('tokens')))

# Built-in function to remove stopwords from our custom list
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered" , stopWords=stopwordList)
df = remover.transform(df)
df = df.select("paperID", "filtered")

# Apply stemming with NLTK
# Built-in class from NLTK
ps = PorterStemmer()
# udf to apply stemming
stemming = udf(lambda x: [ps.stem(item) for item in x], ArrayType(StringType()))
# apply udf to tokens
df = df.withColumn('stemmed', stemming(col('filtered')))
df = df.select("paperID", "stemmed")

# Create a new df to store the paperID and stemmed tokens
paper_terms = df

# Explode/Split the tokens in the list for each paperID and get the distinct tokens
df = df.select("paperID", f.explode("stemmed").alias("tokens"))

#.distinct().orderBy("paperID")

# Assign count of 1 to each token w.r.t. the paperID since the tokens are distinct
df = df.groupBy("paperID","tokens").count()

# Get the number of distinct papers
num_papers = w_df.select("paperID").distinct().count()

# Get the value of ten percent of the number of papers
ten_percent = math.ceil(num_papers*.1)

# Create a new df with the tokens and count (without paperID)
df2 = df.select("tokens", "count")
# Count the number of papers containing the tokens
df2 = df2.groupBy("tokens").agg(f.collect_list("tokens").alias("duplicated_values"), f.sum("count").alias("count"))
# Filter out tokens that appeared in more than 10 percent of the papers
df2 = df2.drop("duplicated_values").orderBy((col("count")).desc()).filter(col("count") < ten_percent)
# Filter out tokens that appeared in less than 20 papers
# Limit the df to 1000 tokens
df2 = df2.filter(col("count") >= 20).limit(1000)

# Create a new df with terms and count
important_words = df2.select(col("tokens").alias("terms"), col("count"))
# Output the set of important words, T
important_words.show(10)

17677
+--------+-----+
|   terms|count|
+--------+-----+
|   motif|   29|
| discuss|   29|
|  improv|   29|
| similar|   29|
|   error|   29|
|properti|   29|
|    test|   29|
|   scale|   28|
| suggest|   28|
|    rate|   28|
+--------+-----+
only showing top 10 rows



In [10]:
# Create a new df where each term is replaced by a unique index that takes a value from the range between 0 and |T| − 1
df = important_words.withColumn("row_num", row_number().over(Window.orderBy("count"))-1)

# Create a df to store the indices and the corresponding terms
terms_index_hash = df.select(col("row_num").alias("index"), "terms")

terms_index_hash.show()

+-----+-----------+
|index|      terms|
+-----+-----------+
|    0|       code|
|    1|      major|
|    2|     author|
|    3|      defin|
|    4|       year|
|    5|    insight|
|    6|bioinformat|
|    7|     captur|
|    8|       rang|
|    9|      build|
|   10|   demonstr|
|   11|      singl|
|   12|     analys|
|   13|     observ|
|   14|      local|
|   15|    potenti|
|   16|   interest|
|   17|    control|
|   18|     articl|
|   19|  scientist|
+-----+-----------+
only showing top 20 rows



In [9]:
# Get the number of distinct terms
num_terms = terms_index_hash.select("terms").distinct().count()

print(num_terms)

91


In [90]:
#Split (explode) the list of words into a column of tokens and
#count the number of occurences of the tokens per paper
#p_terms is a df with paperID, terms, and count
p_terms = paper_terms.select("paperID", f.explode("stemmed").alias("terms"))
#.groupBy("paperID", "terms").count()
#p_terms = p_terms.orderBy("paperID", "count")

# Join p_terms with the terms_index_hash to replace the terms with the indices
joined_df = terms_index_hash.join(p_terms, ["terms"])
# Drop the terms because now they are represented by the indices
joined_df = joined_df.drop("terms")
joined_df  = joined_df.orderBy("paperID")

# Create a new df to compute the term frequency vectors
tf_df = joined_df

tf_df = tf_df.groupby("paperID").agg(f.concat_ws(", ", f.collect_list(tf_df.index)).alias("index"))
tf_df = tf_df.withColumn("terms_index", split(col("index"), ",\s*").cast(ArrayType(StringType())).alias("index"))
tf_df = tf_df.drop("index")
tf_df.show()


# Concatenate the indices and the count (occurences in papers)
# tf_df = tf_df.withColumn("index_count", f.concat(col("index"), lit(", "), col("count")))
# tf_df = tf_df.drop("index", "count")

# # Concatenate the index_counts per paperID ("un-explode")
# tf_df = tf_df.groupby("paperID").agg(f.concat_ws(", ", f.collect_list(tf_df.index_count)).alias("index_count"))
# # Create a new column and casting the index_count into an array with integer type
# # The terms_count of the tf_df column is now a list where the odd positions are the terms indices and the even positions
# tf_df = tf_df.withColumn("terms_count", split(col("index_count"), ",\s*").cast(ArrayType(IntegerType())).alias("terms_count"))
# tf_df = tf_df.drop("index_count")

+-------+--------------------+
|paperID|         terms_index|
+-------+--------------------+
| 159967|[5, 44, 52, 52, 6...|
|2212959|     [4, 44, 72, 72]|
| 333353|[20, 33, 71, 71, ...|
| 438129|[18, 29, 43, 43, ...|
| 166220|       [6, 6, 6, 70]|
|2883810|             [5, 76]|
|1288940|[7, 13, 34, 46, 4...|
|5251453|[2, 2, 2, 2, 12, ...|
|7515828|             [5, 76]|
|2739852|[4, 5, 13, 23, 26...|
|5961524|[16, 24, 33, 38, ...|
|    272|[8, 12, 14, 26, 2...|
|  77265|            [45, 47]|
| 820297|[4, 18, 30, 31, 3...|
|1042553|[15, 17, 17, 17, ...|
|    154|                [46]|
|2883820|                [48]|
|5434882|[13, 18, 19, 19, ...|
|1332540|                [42]|
|1777140|            [52, 54]|
+-------+--------------------+
only showing top 20 rows



In [84]:
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel

# Get the number of distinct terms
num_terms = terms_index_hash.select("terms").distinct().count()

print(num_terms)

cv = CountVectorizer(inputCol="terms_index", outputCol="vectors")
model = cv.fit(tf_df)
x = model.transform(tf_df).show(truncate=False)

91
+-------+------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------+
|paperID|terms_index                                                                                                 |vectors                                                                                                |
+-------+------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------+
|159967 |[5, 44, 52, 52, 63, 79, 84, 87, 90]                                                                         |(91,[0,1,3,7,31,35,49,79],[1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0])                                           |
|2212959|[4, 44, 72, 72]                                                                                 

In [49]:
# udf to get the sparse vector of term frequency
def vector_list_map(x):
    # a list with size of number of terms and fill it with zeroes 

    mylist = [0] * num_terms
    for i in range(0, len(x), 2):
        mylist[int(x[i])] = x[i+1]
        
    # store the indices of non-zero elements
    nonzero_indices = np.nonzero(mylist)[0].tolist()
    # store the value of non-zero elements
    nonzero_counts = [num for num in mylist if num]
    # combine them to make a sparse vector
    sparse = SparseVector(num_terms, nonzero_indices, nonzero_counts)
    return sparse
           
vector_map = udf(lambda x: vector_list_map(x), VectorUDT())

# applying the udf to the terms_count column to create the term_frequency_sparse column
vector_df = tf_df.withColumn('term_frequency_sparse', vector_map(col('terms_count')))

vector_df = vector_df.drop("terms_count")

#vector_df.show(10, truncate=False)

vector_df.where(vector_df.paperID=="80546").show(truncate = False)

AnalysisException: "cannot resolve '`terms_count`' given input columns: [index, paperID];;\n'Project [index#1505, paperID#164, <lambda>('terms_count) AS term_frequency_sparse#1774]\n+- Sort [paperID#164 ASC NULLS FIRST], true\n   +- Project [index#1505, paperID#164]\n      +- Project [terms#1361, index#1505, paperID#164]\n         +- Join Inner, (terms#1361 = terms#1464)\n            :- Project [row_num#1499 AS index#1505, terms#1361]\n            :  +- Project [terms#1361, count#1349L, row_num#1499]\n            :     +- Project [terms#1361, count#1349L, _we0#1500, (_we0#1500 - 1) AS row_num#1499]\n            :        +- Window [row_number() windowspecdefinition(count#1349L ASC NULLS FIRST, ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS _we0#1500], [count#1349L ASC NULLS FIRST]\n            :           +- Project [terms#1361, count#1349L]\n            :              +- Project [tokens#1313 AS terms#1361, count#1349L]\n            :                 +- GlobalLimit 1000\n         

In [11]:
### TF-IDF with built-in function ###

from pyspark.ml.feature import HashingTF, IDF, Tokenizer

idf = IDF(inputCol="term_frequency_sparse", outputCol="features")
idfModel = idf.fit(vector_df)
rescaledData = idfModel.transform(vector_df)
tf_idf_built_in = rescaledData.select("paperID", "features")

tf_idf_built_in.show()

+-------+--------------------+
|paperID|            features|
+-------+--------------------+
| 159967|(64,[6,16,17,21,2...|
|2212959|(64,[6,26,48],[2....|
| 333353|(64,[2,7,10,19,24...|
| 438129|(64,[4,30],[2.542...|
| 166220|(64,[20,29,37,38,...|
|2883810|(64,[24,59],[2.45...|
|1288940|(64,[8,25,39,46,5...|
|5251453|(64,[5,15,22,29,3...|
|7515828|(64,[24,34,59],[2...|
|2739852|(64,[3,9,10,12,14...|
|5961524|(64,[1,2,7,8,24],...|
|    272|(64,[7,17,23,28,4...|
|6573750|(64,[18,55],[2.49...|
|  77265|(64,[2,56],[2.542...|
| 820297|(64,[11,19,35],[2...|
|1042553|(64,[17,21,27,35,...|
|2883820|(64,[8,50,57],[2....|
|5434882|(64,[2,4,29,37,45...|
|1332540|(64,[35],[2.40919...|
|1777140|(64,[16,41,48,49,...|
+-------+--------------------+
only showing top 20 rows



In [12]:
# create the user profile using the tf_idf dataframe and the users' library dataframe
user_profile = tf_idf_built_in.join(user_df, ["paperID"]).orderBy("userID").select('userId', 'features')

# convert the dataframe to RDD to sum up the tf_idf vector of each user and then convert back to dataframe
user_profile = user_profile.rdd.mapValues(lambda v: v.toArray()).reduceByKey(lambda x, y: x + y).mapValues(lambda x: DenseVector(x))\
.toDF(["userId", "features"])

user_profile = user_profile.select("userId", col("features").alias("user_profile"))


In [13]:
def to_sparse(x):        
    # store the indices of non-zero elements
    nonzero_indices = np.nonzero(x)[0].tolist()
    # store the value of non-zero elements
    nonzero_counts = [num for num in x if num]
    # combine them to make a sparse vector
    sparse = SparseVector(num_terms, nonzero_indices, nonzero_counts)
    return sparse

to_sparse_udf = udf(lambda x: to_sparse(x), VectorUDT())

user_profile = user_profile.withColumn("user_profile", to_sparse_udf(col("user_profile")))

user_profile.show()

+--------------------+--------------------+
|              userId|        user_profile|
+--------------------+--------------------+
|d503571e44a0373eb...|(64,[2,10,12,18,2...|
|f1e1cd4ff25018273...|(64,[3,8,20,29,38...|
|bbcd9dae3160ddcb9...|(64,[3,12,19,29,3...|
|a0bbf6bb9b1c818f3...|(64,[29,32,60],[2...|
|1eac022a97d683eac...|(64,[1,4,10,11,13...|
|cf9c7f356092c34be...|(64,[1,9,10,14,18...|
|3b715ebaf1f8f81a1...|(64,[3,6,9,10,14,...|
|b36c3189bb1457cd0...|(64,[24],[2.45175...|
|f3c28e50db4ce8ad8...|(64,[2,10,12,18,2...|
|b656009a6efdc8b1a...|(64,[0,2,4,5,9,10...|
|d85f7d83f27b3f533...|(64,[0,8,24,29,44...|
|f05bcffe7951de9e5...|(64,[1,3,4,7,10,1...|
|4c8912d1b04471cf5...|(64,[1,4,10,11,13...|
+--------------------+--------------------+



In [14]:
df = unrated_df.join(tf_idf_built_in, ["paperID"]).join(user_profile, ["userID"])

df = df.select("userID", "paperID", "user_profile", col("features").alias("paper_profile"))

df.show()

+--------------------+-------+--------------------+--------------------+
|              userID|paperID|        user_profile|       paper_profile|
+--------------------+-------+--------------------+--------------------+
|f1e1cd4ff25018273...| 166220|(64,[3,8,20,29,38...|(64,[20,29,37,38,...|
|f1e1cd4ff25018273...| 820297|(64,[3,8,20,29,38...|(64,[11,19,35],[2...|
|f1e1cd4ff25018273...|1042553|(64,[3,8,20,29,38...|(64,[17,21,27,35,...|
|f1e1cd4ff25018273...|3010240|(64,[3,8,20,29,38...|(64,[29,31],[2.45...|
|f1e1cd4ff25018273...| 523772|(64,[3,8,20,29,38...|(64,[35],[2.40919...|
|f1e1cd4ff25018273...|1279898|(64,[3,8,20,29,38...|(64,[24,33,51,56]...|
|f1e1cd4ff25018273...| 920055|(64,[3,8,20,29,38...|(64,[24,38,59],[2...|
|f1e1cd4ff25018273...| 965334|(64,[3,8,20,29,38...|(64,[2,51,54,59],...|
|f1e1cd4ff25018273...|     99|(64,[3,8,20,29,38...|(64,[7,17,18,20,2...|
|f1e1cd4ff25018273...|7355647|(64,[3,8,20,29,38...|(64,[3,12,41,43,4...|
|f1e1cd4ff25018273...|   4280|(64,[3,8,20,29,38...|

In [15]:
df = unrated_df.join(tf_idf_built_in, ["paperID"]).join(user_profile, ["userID"])

df = df.select("userID", "paperID", "user_profile", col("features").alias("paper_profile"))

# def to_dense(x):
#     return DenseVector(x.toArray())

# to_dense_udf = udf(lambda x: to_dense(x), VectorUDT())

# df = df.withColumn('paper_profile_dense', to_dense_udf(df.paper_profile))
# df = df.drop("paper_profile")

df_selected = df.where(df.userID=="1eac022a97d683eace8815545ce3153f")

df_selected.show()

+--------------------+--------+--------------------+--------------------+
|              userID| paperID|        user_profile|       paper_profile|
+--------------------+--------+--------------------+--------------------+
|1eac022a97d683eac...|  166220|(64,[1,4,10,11,13...|(64,[20,29,37,38,...|
|1eac022a97d683eac...|  820297|(64,[1,4,10,11,13...|(64,[11,19,35],[2...|
|1eac022a97d683eac...| 1279898|(64,[1,4,10,11,13...|(64,[24,33,51,56]...|
|1eac022a97d683eac...|  920055|(64,[1,4,10,11,13...|(64,[24,38,59],[2...|
|1eac022a97d683eac...|  965334|(64,[1,4,10,11,13...|(64,[2,51,54,59],...|
|1eac022a97d683eac...|      99|(64,[1,4,10,11,13...|(64,[7,17,18,20,2...|
|1eac022a97d683eac...| 7355647|(64,[1,4,10,11,13...|(64,[3,12,41,43,4...|
|1eac022a97d683eac...|    4280|(64,[1,4,10,11,13...|(64,[24],[2.45175...|
|1eac022a97d683eac...| 4302361|(64,[1,4,10,11,13...|(64,[29],[2.45175...|
|1eac022a97d683eac...|   72879|(64,[1,4,10,11,13...|(64,[24,47,50,60]...|
|1eac022a97d683eac...|   90558|(64,[1,

In [49]:
import numpy as np 

def cos_sim(u, p):
    result = (np.dot(u, p))/(np.linalg.norm(u) * np.linalg.norm(p))
    result = result.item()
    return result

compute_sim = udf(cos_sim, FloatType())

def cbrs_tf_idf(u, k):
    sim_df = u.withColumn('Similarity', compute_sim(u.user_profile, u.paper_profile))
    window = Window.partitionBy(col("userID")).orderBy((col("Similarity")).desc())
    sim_df = sim_df.select(col('*'), row_number().over(window).alias('row_number')).where(col('row_number') <= k)
    get_r = sim_df.select("userID", "paperID", col("row_number").alias("rank"))
    cbrs = get_r.select("userID", "paperID")
    cbrs = cbrs.groupby("userID").agg(f.concat_ws(", ", f.collect_list(cbrs.paperID)).alias("top_papers"))
    
    print("The top " + str(k) + " papers for " + str(cbrs.head()[0]) + " are " + str(cbrs.head()[1]))

cbrs_tf_idf(df_selected, 5)

The top 5 papers for 1eac022a97d683eace8815545ce3153f are 100088, 211804, 557229, 478707, 4302361


In [18]:
### LDA ###
from pyspark.ml.clustering import LDA

# just rename the column to use it with the built-in methods
termFrequencyVector = vector_df.select('paperId', col('term_frequency_sparse').alias('features'))
# Trains a LDA model
# set k=40 to have 40 different topics
lda = LDA(k=40)
model = lda.fit(termFrequencyVector)

# each topic is described by 5 terms
topics = model.describeTopics(5)

# print("The topics described by their top-weighted terms:")
#topics.show(truncate=False)

# Shows the result
# it shows the probabilty of each topic for each paper
transformed = model.transform(termFrequencyVector)
transformed.show()

+-------+--------------------+--------------------+
|paperId|            features|   topicDistribution|
+-------+--------------------+--------------------+
| 159967|(64,[6,16,17,21,2...|[9.45110635789367...|
|2212959|(64,[6,26,48],[1....|[0.00441872016704...|
| 333353|(64,[2,7,10,19,24...|[0.56343393574946...|
| 438129|(64,[4,30],[1.0,1...|[0.00885841139294...|
| 166220|(64,[20,29,37,38,...|[0.00240762022449...|
|2883810|(64,[24,59],[1.0,...|[0.00885841139294...|
|1288940|(64,[8,25,39,46,5...|[0.26311822294429...|
|5251453|(64,[5,15,22,29,3...|[0.22258093015662...|
|7515828|(64,[24,34,59],[1...|[0.00663593507796...|
|2739852|(64,[3,9,10,12,14...|[0.00110272213561...|
|5961524|(64,[1,2,7,8,24],...|[0.90392588951444...|
|    272|(64,[7,17,23,28,4...|[0.00176498105827...|
|6573750|(64,[18,55],[1.0,...|[0.00885841139294...|
|  77265|(64,[2,56],[1.0,1...|[0.67868904983830...|
| 820297|(64,[11,19,35],[1...|[0.00663593507796...|
|1042553|(64,[17,21,27,35,...|[0.00155716299784...|
|2883820|(64

In [33]:
# creating a user profile based on the LDA results

lda_user_profile = transformed.join(user_df, ["paperID"]).orderBy("userID").select('userId', col('topicDistribution').alias('features'))

lda_user_profile = lda_user_profile.rdd.mapValues(lambda v: v.toArray()).reduceByKey(lambda x, y: x + y).mapValues(lambda x: DenseVector(x))\
.toDF(["userId", "features"])
lda_user_profile = lda_user_profile.select("userId", col("features").alias("lda_user_profile"))

lda_user_profile = lda_user_profile.withColumn("user_profile", to_sparse_udf(col("lda_user_profile")))

lda_user_profile.show()

+--------------------+--------------------+--------------------+
|              userId|    lda_user_profile|        user_profile|
+--------------------+--------------------+--------------------+
|d503571e44a0373eb...|[0.00189117886967...|(64,[0,1,2,3,4,5,...|
|f1e1cd4ff25018273...|[1.07412582463749...|(64,[0,1,2,3,4,5,...|
|bbcd9dae3160ddcb9...|[0.52461795827945...|(64,[0,1,2,3,4,5,...|
|a0bbf6bb9b1c818f3...|[0.00331207990297...|(64,[0,1,2,3,4,5,...|
|1eac022a97d683eac...|[2.07879795154980...|(64,[0,1,2,3,4,5,...|
|cf9c7f356092c34be...|[0.00189117886967...|(64,[0,1,2,3,4,5,...|
|3b715ebaf1f8f81a1...|[0.26412840422374...|(64,[0,1,2,3,4,5,...|
|b36c3189bb1457cd0...|[0.01331922332444...|(64,[0,1,2,3,4,5,...|
|f3c28e50db4ce8ad8...|[0.00189117886967...|(64,[0,1,2,3,4,5,...|
|b656009a6efdc8b1a...|[0.93239989082039...|(64,[0,1,2,3,4,5,...|
|d85f7d83f27b3f533...|[0.25812162666935...|(64,[0,1,2,3,4,5,...|
|f05bcffe7951de9e5...|[0.03399150910360...|(64,[0,1,2,3,4,5,...|
|4c8912d1b04471cf5...|[0.

In [51]:
df_lda = unrated_df.join(tf_idf_built_in, ["paperID"]).join(lda_user_profile, ["userID"])

df_lda = df_lda.select("userID", "paperID", "user_profile", col("features").alias("paper_profile"))

#df_lda.show()

In [35]:
df_selected_lda = df_lda.where(df_lda.userID=="1eac022a97d683eace8815545ce3153f")

df_selected_lda.show()

+--------------------+--------+--------------------+--------------------+
|              userID| paperID|        user_profile|       paper_profile|
+--------------------+--------+--------------------+--------------------+
|1eac022a97d683eac...|  166220|(64,[0,1,2,3,4,5,...|(64,[20,29,37,38,...|
|1eac022a97d683eac...|  820297|(64,[0,1,2,3,4,5,...|(64,[11,19,35],[2...|
|1eac022a97d683eac...| 1279898|(64,[0,1,2,3,4,5,...|(64,[24,33,51,56]...|
|1eac022a97d683eac...|  920055|(64,[0,1,2,3,4,5,...|(64,[24,38,59],[2...|
|1eac022a97d683eac...|  965334|(64,[0,1,2,3,4,5,...|(64,[2,51,54,59],...|
|1eac022a97d683eac...|      99|(64,[0,1,2,3,4,5,...|(64,[7,17,18,20,2...|
|1eac022a97d683eac...| 7355647|(64,[0,1,2,3,4,5,...|(64,[3,12,41,43,4...|
|1eac022a97d683eac...|    4280|(64,[0,1,2,3,4,5,...|(64,[24],[2.45175...|
|1eac022a97d683eac...| 4302361|(64,[0,1,2,3,4,5,...|(64,[29],[2.45175...|
|1eac022a97d683eac...|   72879|(64,[0,1,2,3,4,5,...|(64,[24,47,50,60]...|
|1eac022a97d683eac...|   90558|(64,[0,

In [50]:
cbrs_tf_idf(df_selected_lda, 5)

The top 5 papers for 1eac022a97d683eace8815545ce3153f are 849862, 211804, 249, 664041, 81501


## Exercise 4. 4 Sampling and data preparation

In [52]:
sampled_user_profile = user_profile.sample(False, 1).limit(10)
sampled_user_profile.show()

Py4JError: An error occurred while calling o605.sample. Trace:
py4j.Py4JException: Method sample([class java.lang.Boolean, class java.lang.Integer, class java.lang.Long]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:272)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)

